In [ ]:
import numpy as np
import pandas as pd

timepoint_data = np.zeros((4, 3, 3))


timepoints = 'baseline', '1 minute', '1 hour', '1 day'
timepoints_as_delta = [pd.Timedelta(0, 'm'), pd.Timedelta(1, 'm'), pd.Timedelta(1, 'h'), pd.Timedelta(1, 'd')]
#timepoint_colors = ['mistyrose', 'salmon', 'indianred', 'darkred']
timepoint_colors = ['lightblue', 'cadetblue', 'deepskyblue', 'darkcyan']
    
dataframes = [pd.DataFrame(d) for d in timepoint_data]
data_df = pd.concat(dataframes, keys=timepoints_as_delta, names=['timepoint'])
data_df.index.names = ['timepoint', 'Y']
data_df.columns.name = 'X'

def colorize(x):
    t = x.reset_index()['timepoint']
    color_map = dict(zip(timepoints_as_delta, timepoint_colors))
    colors = [color_map[t] for t in t]
    return [f'background-color: {c}; color: {c}' for c in colors]

data_df[:] = 'text'
data_df.style.apply(colorize)

In [ ]:
def panel():
    return np.array([
        [0, 0, 0], 
        [1, 0, 0], 
        [1, 1, 0], 
        [0, 1, 0]]
    )


def rotate(a, axis, theta):
    l = 0
    m = 0
    n = 0
    if axis == 'x':
        l = 1
    elif axis == 'y':
        m = 1
    elif axis == 'z':
        n = 1
    
    m = np.array([
        [l*l*(1-np.cos(theta))+np.cos(theta), 
         m*l*(1-np.cos(theta))-n*np.sin(theta),
         n*l*(1-np.cos(theta))+m*np.sin(theta),
         0,
        ],
        [l*m*(1-np.cos(theta))+n*np.sin(theta), 
         m*m*(1-np.cos(theta))+np.cos(theta),
         n*m*(1-np.cos(theta))-l*np.sin(theta),
         0,
        ],
        [l*n*(1-np.cos(theta))-m*np.sin(theta), 
         m*n*(1-np.cos(theta))+l*np.sin(theta),
         n*n*(1-np.cos(theta))+np.cos(theta),
         0,
        ],
        [0, 0, 0, 1]
    ])
    a = np.pad(a, ((0, 0), (0, 1)), mode='constant', constant_values=1)
    a = a.T
    result = m @ a
    return result.T[:, :-1]


def translate(a, axis, delta):
    x = 0
    y = 0
    z = 0
    if axis == 'x':
        x = delta
    elif axis == 'y':
        y = delta
    elif axis == 'z':
        z = delta
    
    m = np.array([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])
    a = np.pad(a, ((0, 0), (0, 1)), mode='constant', constant_values=1)
    a = a.T
    result = m @ a
    return result.T[:, :-1]

In [ ]:
def create_cube(x, y, z, color):
    panels = [
        panel(),
        translate(panel(), 'z', 1),
        rotate(panel(), 'x', np.pi/2),
        translate(rotate(panel(), 'x', np.pi/2), 'y', 1),
        rotate(panel(), 'y', -np.pi/2),
        translate(rotate(panel(), 'y', -np.pi/2), 'x', 1),
    ]
    
    def move_panel(p, x, y, z):
        p = translate(p, 'x', x)
        p = translate(p, 'y', y)
        p = translate(p, 'z', z)
        return p
    
    panels = [move_panel(p, x, y, z) for p in panels]
    collection = Poly3DCollection(panels, facecolors=color,
                                  linewidths=1, edgecolors='white', 
                                  alpha=1)
    return collection


import pylab as pl
figure = pl.gcf()
ax = Axes3D(fig=figure)


for z, (timepoint, color) in enumerate(zip(timepoints, timepoint_colors)):
    for x in range(5):
        for y in range(5):
            ax.add_collection3d(create_cube(x, z, y, color))
    ax.text3D(5.5, z, 0, timepoint, color=color)

ax.auto_scale_xyz([0, 5], [0, 5], [0, 5])
#ax.xaxis.set_ticks([])
#ax.yaxis.set_ticks([])
#ax.zaxis.set_ticks([])
ax.axis('off')

In [ ]:
%matplotlib inline
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
import matplotlib.pyplot as plt

points = np.array([[-1, -1, -1],
                  [1, -1, -1 ],
                  [1, 1, -1],
                  [-1, 1, -1],
                  [-1, -1, 1],
                  [1, -1, 1 ],
                  [1, 1, 1],
                  [-1, 1, 1]])

P = [[2.06498904e-01 , -6.30755443e-07 ,  1.07477548e-03],
 [1.61535574e-06 ,  1.18897198e-01 ,  7.85307721e-06],
 [7.08353661e-02 ,  4.48415767e-06 ,  2.05395893e-01]]

Z = np.zeros((8,3))
for i in range(8): Z[i,:] = np.dot(points[i,:],P)
Z = 10.0*Z

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

r = [-1,1]

X, Y = np.meshgrid(r, r)
# plot vertices
ax.scatter3D(Z[:, 0], Z[:, 1], Z[:, 2], '.')

# list of sides' polygons of figure
verts = [
     [Z[0],Z[1],Z[2],Z[3]],
     [Z[4],Z[5],Z[6],Z[7]], 
     [Z[0],Z[1],Z[5],Z[4]], 
     [Z[2],Z[3],Z[7],Z[6]], 
     [Z[1],Z[2],Z[6],Z[5]],
     [Z[4],Z[7],Z[3],Z[0]], 
     [Z[2],Z[3],Z[7],Z[6]]]

# plot sides
#ax.add_collection3d(Poly3DCollection(verts, 
# facecolors='seagreen', linewidths=1, edgecolors='olive', alpha=.25))

ax.add_collection3d(Poly3DCollection([panel], 
 facecolors='red', linewidths=1, edgecolors='olive', alpha=.25))

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.axis('off')

plt.show()

In [ ]:
data = np.zeros((2, 3, 5, 5))

timepoints = 'before', 'after'
treatments = '0.1 ml', '0.2 ml', '0.4 ml'

dataframes = []
keys = []

for timepoint, timepoint_data in zip(timepoints, data):
    for treatment, treatment_data in zip(treatments, timepoint_data):
        df = pd.DataFrame(treatment_data)
        dataframes.append(df)
        keys.append((timepoint, treatment))
        
data_df = pd.concat(dataframes, keys=keys, names=['timepoint', 'treatment'])
data_df